In [63]:
import pandas as pd
import numpy as np

In [64]:
prices = pd.read_csv('airbnb_price.csv')
room_types = pd.read_excel('airbnb_room_type.xlsx')
reviews = pd.read_csv('airbnb_last_review.tsv', sep='\t')

display(prices.head())
display(room_types.head())
display(reviews.head())

,listing_id,price,nbhood_full
0,2595,225 dollars,"Manhattan, Midtown"
1,3831,89 dollars,"Brooklyn, Clinton Hill"
2,5099,200 dollars,"Manhattan, Murray Hill"
3,5178,79 dollars,"Manhattan, Hell's Kitchen"
4,5238,150 dollars,"Manhattan, Chinatown"


,listing_id,description,room_type
0,2595,Skylit Midtown Castle,Entire home/apt
1,3831,Cozy Entire Floor of Brownstone,Entire home/apt
2,5099,Large Cozy 1 BR Apartment In Midtown East,Entire home/apt
3,5178,Large Furnished Room Near B'way,private room
4,5238,Cute & Cozy Lower East Side 1 bdrm,Entire home/apt


,listing_id,host_name,last_review
0,2595,Jennifer,May 21 2019
1,3831,LisaRoxanne,July 05 2019
2,5099,Chris,June 22 2019
3,5178,Shunichi,June 24 2019
4,5238,Ben,June 09 2019


### Cleaning the price columns

In [65]:
prices['price'] = prices['price'].str.replace(' dollars', '')
prices['price'] = pd.to_numeric(prices['price'])

display(prices['price'].describe())

count    25209.000000
mean       141.777936
std        147.349137
min          0.000000
25%         69.000000
50%        105.000000
75%        175.000000
max       7500.000000
Name: price, dtype: float64

### Caculating average price

In [66]:
free_listing = prices['price'] == 0

prices = prices.loc[~free_listing]
avg_price = round(prices['price'].mean(), 2)

print('The average price per night for an Airbnb listing in NYC is ${}'.format(avg_price))

The average price per night for an Airbnb listing in NYC is $141.82


### Compare costs to the private rental market 

In [67]:
prices['price_per_month'] = prices['price']*365/12

avg_price_per_month = round(prices['price_per_month'].mean(), 2)

print('Airbnb monthly cost ${}, while in the private market you would pay ${}'.format(avg_price_per_month, '3100.00'))

Airbnb monthly cost $4313.61, while in the private market you would pay $3100.00


### Cleaning the room type column 

In [68]:
room_types['room_type'] = room_types['room_type'].str.lower()
room_types['room_type'] = room_types['room_type'].astype('category')

room_types['room_type'].value_counts()

entire home/apt    13266
private room       11356
shared room          587
Name: room_type, dtype: int64

### What timeframe are we working with?

In [69]:
reviews['last_review'] = pd.to_datetime(reviews['last_review'])

first_reviews = reviews['last_review'].dt.date.min()
last_reviews = reviews['last_review'].dt.date.max()

print('The lastest review is {}, the earliest review is {}'.format(first_reviews,last_reviews))

The lastest review is 2019-01-01, the earliest review is 2019-07-09


### Joining the DataFrames 

In [70]:
room_and_prices = prices.merge(room_types, how='outer', on='listing_id')
airbnb_merge = room_and_prices.merge(reviews, how='outer', on='listing_id')

airbnb_merge.dropna(inplace=True)

In [71]:
airbnb_merge

,listing_id,price,nbhood_full,price_per_month,description,room_type,host_name,last_review
0,2595,225.0,"Manhattan, Midtown",6843.750000,Skylit Midtown Castle,entire home/apt,Jennifer,2019-05-21
1,3831,89.0,"Brooklyn, Clinton Hill",2707.083333,Cozy Entire Floor of Brownstone,entire home/apt,LisaRoxanne,2019-07-05
2,5099,200.0,"Manhattan, Murray Hill",6083.333333,Large Cozy 1 BR Apartment In Midtown East,entire home/apt,Chris,2019-06-22
3,5178,79.0,"Manhattan, Hell's Kitchen",2402.916667,Large Furnished Room Near B'way,private room,Shunichi,2019-06-24
4,5238,150.0,"Manhattan, Chinatown",4562.500000,Cute & Cozy Lower East Side 1 bdrm,entire home/apt,Ben,2019-06-09
...,...,...,...,...,...,...,...,...
25197,36425863,129.0,"Manhattan, Upper East Side",3923.750000,Lovely Privet Bedroom with Privet Restroom,private room,Rusaa,2019-07-07
25198,36427429,45.0,"Queens, Flushing",1368.750000,No.2 with queen size bed,private room,H Ai,2019-07-07
25199,36438336,235.0,"Staten Island, Great Kills",7147.916667,Seas The Moment,private room,Ben,2019-07-07
25200,36442252,100.0,"Bronx, Mott Haven",3041.666667,1B-1B apartment near by Metro,entire home/apt,Blaine,2019-07-07


### Analyzing listing prices by NYC borough

In [72]:
airbnb_merge['borough'] = airbnb_merge['nbhood_full'].str.partition(',')[0]

boroughs = airbnb_merge.groupby('borough')['price'].agg(['sum','mean','median','count'])
boroughs = boroughs.round(2).sort_values('mean', ascending=False)

display(boroughs)

,sum,mean,median,count
borough,,,,
Manhattan,1898417.0,184.04,149.0,10315
Brooklyn,1275250.0,122.02,95.0,10451
Queens,320715.0,92.83,70.0,3455
Staten Island,22974.0,86.04,71.0,267
Bronx,55156.0,79.25,65.0,696


### Price range by borough

In [95]:
label_names = ['Budget','Average','Expensive','Extravagant']
ranges = [0, 69, 175, 350, np.inf]

airbnb_merge['price_range'] = pd.cut(airbnb_merge['price'], bins=ranges, labels=label_names)
price_by_borough = airbnb_merge.groupby(['borough','price_range'])['price_range'].count()
display(price_by_borough.unstack())

price_range,Budget,Average,Expensive,Extravagant
borough,,,,
Bronx,381,285,25,5
Brooklyn,3194,5532,1466,259
Manhattan,1148,5285,3072,810
Queens,1631,1505,291,28
Staten Island,124,123,20,0


In [92]:
airbnb_merge[airbnb_merge['price'] == 175]

,listing_id,price,nbhood_full,price_per_month,description,room_type,host_name,last_review,borough,price_range
77,23135,175.0,"Brooklyn, Carroll Gardens",5322.916667,House On Henry (3rd FLR Suite),entire home/apt,Annette,2019-06-24,Brooklyn,Average
182,62891,175.0,"Manhattan, East Village",5322.916667,Smallest House In The Village,entire home/apt,Olivia,2019-05-24,Manhattan,Average
257,94477,175.0,"Brooklyn, Bedford-Stuyvesant",5322.916667,The Vernon On Greene,entire home/apt,Jason,2019-07-05,Brooklyn,Average
271,107630,175.0,"Brooklyn, Greenpoint",5322.916667,Sweet Historic Greenpoint Duplex,entire home/apt,Maya,2019-06-20,Brooklyn,Average
304,141984,175.0,"Manhattan, Nolita",5322.916667,Charming Nolita Apartment!!,entire home/apt,Vanessa,2019-06-10,Manhattan,Average
...,...,...,...,...,...,...,...,...,...,...
24861,35707367,175.0,"Manhattan, Chelsea",5322.916667,Great studio located in Heart of Chelsea,entire home/apt,Ezequiel,2019-06-30,Manhattan,Average
24900,35739927,175.0,"Manhattan, West Village",5322.916667,"Charming ""Library"" Apt, Heart of Greenwich Vil...",entire home/apt,Evelyn,2019-06-21,Manhattan,Average
24911,35758732,175.0,"Brooklyn, Bedford-Stuyvesant",5322.916667,"The Gallery Bed & Breakfast in Bed-Stuy, Brooklyn",private room,The Gallery BnB,2019-07-07,Brooklyn,Average
24988,35865788,175.0,"Queens, Long Island City",5322.916667,Brand new 1Bedroom apartment with private terrace,entire home/apt,Kasper,2019-06-23,Queens,Average
